### 📝 Lab #3 (Part 1 응용): 최고의 앙상블 기법 찾기

지금까지 배운 4가지 앙상블 기법(Voting, Bagging, Boosting, Stacking)을 모두 사용하여, 심부전 예측 문제에서 어떤 기법이 가장 높은 성능을 보이는지 비교 분석하는 미니 프로젝트를 수행합니다.

#### 과제 목표

1.  Voting, Bagging, AdaBoost, Stacking 분류기를 각각 생성하고 학습시킵니다.
    * **Voting**: Soft Voting 방식을 사용합니다.
    * **Bagging**: 결정 트리를 기본 모델로 사용하고, `n_estimators=100`으로 설정합니다.
    * **AdaBoost**: `n_estimators=100`, `learning_rate=0.1`로 설정합니다.
    * **Stacking**: 기본 모델은 `LogisticRegression`과 `KNeighborsClassifier`를 사용하고, 메타 모델은 `DecisionTreeClassifier`를 사용합니다.
2.  각 모델의 **정확도(Accuracy)**와 **ROC-AUC 점수**를 계산하여 성능을 비교합니다.
3.  `Plotly Express`를 사용하여 각 모델의 성능 지표를 **막대 그래프**로 시각화하고, 어떤 모델이 가장 우수한지 분석합니다.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier, BaggingClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
import plotly.express as px

# 데이터 불러오기
path = '../../datasets/ml/heart-failure/heart_failure_clinical_records_dataset.csv'
df = pd.read_csv(path)

# 특성과 타겟 분리
X = df.drop('DEATH_EVENT', axis=1)
y = df['DEATH_EVENT']

# 학습/테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 1. Voting Classifier (Soft Voting)
voting_clf = VotingClassifier(
    estimators=[
        ('lr', LogisticRegression(max_iter=1000, random_state=42)),
        ('knn', KNeighborsClassifier()),
        ('dt', DecisionTreeClassifier(random_state=42))
    ],
    voting='soft'
)
voting_clf.fit(X_train, y_train)
y_pred_voting = voting_clf.predict(X_test)
y_proba_voting = voting_clf.predict_proba(X_test)[:,1]
acc_voting = accuracy_score(y_test, y_pred_voting)
roc_voting = roc_auc_score(y_test, y_proba_voting)

# 2. Bagging Classifier (DecisionTree, n_estimators=100)
bagging_clf = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(random_state=42),
    n_estimators=100,
    random_state=42
)
bagging_clf.fit(X_train, y_train)
y_pred_bagging = bagging_clf.predict(X_test)
y_proba_bagging = bagging_clf.predict_proba(X_test)[:,1]
acc_bagging = accuracy_score(y_test, y_pred_bagging)
roc_bagging = roc_auc_score(y_test, y_proba_bagging)

# 3. AdaBoost Classifier (n_estimators=100, learning_rate=0.1)
adaboost_clf = AdaBoostClassifier(
    n_estimators=100,
    learning_rate=0.1,
    random_state=42
)
adaboost_clf.fit(X_train, y_train)
y_pred_adaboost = adaboost_clf.predict(X_test)
y_proba_adaboost = adaboost_clf.predict_proba(X_test)[:,1]
acc_adaboost = accuracy_score(y_test, y_pred_adaboost)
roc_adaboost = roc_auc_score(y_test, y_proba_adaboost)

# 4. Stacking Classifier (base: LogisticRegression, KNN / meta: DecisionTree)
stacking_clf = StackingClassifier(
    estimators=[
        ('lr', LogisticRegression(max_iter=1000, random_state=42)),
        ('knn', KNeighborsClassifier())
    ],
    final_estimator=DecisionTreeClassifier(random_state=42),
    passthrough=False,
    cv=5
)
stacking_clf.fit(X_train, y_train)
y_pred_stacking = stacking_clf.predict(X_test)
y_proba_stacking = stacking_clf.predict_proba(X_test)[:,1]
acc_stacking = accuracy_score(y_test, y_pred_stacking)
roc_stacking = roc_auc_score(y_test, y_proba_stacking)

# 결과 정리
results = pd.DataFrame({
    'Model': ['Voting', 'Bagging', 'AdaBoost', 'Stacking'],
    'Accuracy': [acc_voting, acc_bagging, acc_adaboost, acc_stacking],
    'ROC-AUC': [roc_voting, roc_bagging, roc_adaboost, roc_stacking]
})

# Plotly로 시각화
fig_acc = px.bar(results, x='Model', y='Accuracy', title='앙상블 모델별 정확도(Accuracy)', text='Accuracy', color='Model')
fig_roc = px.bar(results, x='Model', y='ROC-AUC', title='앙상블 모델별 ROC-AUC', text='ROC-AUC', color='Model')

fig_acc.show()
fig_roc.show()
